## Image classification with a convolutional neural network

This example is using the cifar10 dataset of keras (https://keras.io/datasets/). The dataset provides 50k training and 10k testing images of a small size and low resolution suitable for low performance environments. The images are in color and there are 10 target classes. The objective is to correctly predict as many images as possible with an convolutional neural network.

### Load the cifar10 dataset and explore the images

In [ ]:
# Importing the necessary libraries
import keras
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
%matplotlib inline

In [ ]:
# Load the preshuffled train and test data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Visualize some images and their labels (source: https://www.cs.toronto.edu/~kriz/cifar.html)
cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

fig = plt.figure(figsize=(20, 6))
for i in range(36):
    ax = fig.add_subplot(3, 12, i + 1, xticks=[], yticks=[])
    ax.set_title("%s" % cifar10_labels[y_train[i][0]])
    ax.imshow(np.squeeze(x_train[i]))

Every image consists of a 3-dimensional vector with the shape (32,32,3). The first two dimensions are corresponding to width and height of the image while the third dimension is holding the values of the three color channels.

In [ ]:
# Display the individual channels of a sample
fig, axes = plt.subplots(1, 4, figsize=(12, 10))
axes[0].imshow(x_train[2])
axes[0].set_title('Original (RGB)\n%s' % str(x_train[2].shape))

for i, l in zip(range(3), ['Red channel','Green channel','Blue channel']): 
    channel=np.zeros((32,32,3))
    channel[:,:,i] = x_train[2][:,:,i]
    axes[i+1].imshow(channel)
    axes[i+1].set_title('%s\n(32, 32, 1)' % (l))
plt.show()

In [ ]:
# Normalize the three color values from 0-255 to 0-1
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [ ]:
# Encode the labels into categorical values (one-hot encoding)
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Break training set into training and validation sets
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

# Print shape of training set
print('x_train shape:', x_train.shape)

# print number of training, validation, and test images
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_valid.shape[0], 'validation samples')

### TODO: Define the architecture for the convolutional neural network

TODO: 
*(Nothing new if you solved the mnist exercise)* You have to define the architecture of the layers. You will need convolutional layers (https://keras.io/layers/convolutional/#conv2d) to handle spatial information of the pixels. Some hints are:
* Stick with the sequential model (there is no temporal information here)
* It is advised to follow a convolutional layer (Conv2D) by a pooling layer (MaxPooling2D)
* If your accuracy is good but your validation accuracy is not, try to prevent overfitting by adding a dropout layer (https://keras.io/layers/core/#dropout)
* To rectifier unit has proven to be a good start for the activation of layers in neural networks (https://keras.io/activations/#available-activations)
* Because you try to predict multiple classes, the final layer needs to be a dense layer with the same amout of neurons as classes to predict. Then use a softmax activation function (http://dataaspirant.com/2017/03/07/difference-between-softmax-function-and-sigmoid-function/) to predict the final class

In [ ]:
# Define the architecture for the neural network
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
# TODO: Add the necessary layers
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

In [ ]:
# Print summary of the model to examine parameters to train
model.summary()

Reminder: The 'Trainable params' shows you the number of parameters the network will have to train. The more parameters, the slower the network learns.

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint   
import livelossplot as lp

plot_learning = lp.PlotLossesKeras()

# train the model
checkpointer = ModelCheckpoint(filepath='image.model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)

callbacks = [checkpointer, plot_learning]

TODO: In the next cell you should train (fit) your network. Try do experiment with different values for the *batch_size* and *epochs* parameters.

In [ ]:
# Fit the model by calling the fit method and supply the necessary
# parameters including the validation data
hist = model.fit(x_train, y_train, batch_size=100, epochs=50,
          validation_data=(x_valid, y_valid), callbacks=callbacks, 
          verbose=1)

If you have trained your model you can load the weights and evaluate your model by executing the following cell. You do not have to change any code.

In [ ]:
# Load the weights that yielded the best validation accuracy
model.load_weights('image.model.weights.best.hdf5')

# Evaluate and print test accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])

If you are satisfied with the accuracy of your model you can use it to make the predictions on the test and visualize some samples by executing the next cell.

In [ ]:
# Make the predictions on the test set
y_hat = model.predict(x_test)

# Plot a random sample of test images, their predicted labels, and ground truth
fig = plt.figure(figsize=(20, 8))
for i, idx in enumerate(np.random.choice(x_test.shape[0], size=32, replace=False)):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx = np.argmax(y_hat[idx])
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(cifar10_labels[pred_idx], cifar10_labels[true_idx]),
                 color=("green" if pred_idx == true_idx else "red"))

Congratulations! How did you do with the classification of the images? Did you get an accuracy of .7 and more?